In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm as tqdm
import os

In [2]:
def parse_ratings_file(file_path):
    data = []
    current_review = {}

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            if line:  # If the line is not empty
                if ': ' in line:  # Check if the line contains the delimiter ': '
                    key, value = line.split(': ', 1)  # Split by the first occurrence of ': '
                    current_review[key] = value
                else:
                    # Handle lines that do not follow the expected key-value format
                    # For example, this could happen for text reviews that span multiple lines
                    if 'text' in current_review:
                        current_review['text'] += ' ' + line  # Append to the 'text' field
                    else:
                        current_review['text'] = line  # Create the 'text' field
            else:  # Blank line indicates the end of a review
                if current_review:  # Ensure that the review is not empty
                    data.append(current_review)
                current_review = {}  # Reset for the next review

        if current_review:  # Catch the last review if there's no trailing blank line
            data.append(current_review)

    # Convert the list of dictionaries to a DataFrame
    return pd.DataFrame(data)

In [3]:
#importing data
BA_path = 'baseData/BeerAdvocate/'
RB_path = 'baseData/RateBeer/'

BA_beers = pd.read_csv(BA_path + 'beers.csv')
BA_breweries = pd.read_csv(BA_path + 'breweries.csv')
BA_users = pd.read_csv(BA_path + 'users.csv')

RB_beers = pd.read_csv(RB_path + 'beers.csv')
RB_breweries = pd.read_csv(RB_path + 'breweries.csv')
RB_users = pd.read_csv(RB_path + 'users.csv')


In [47]:
BA_ratings = parse_ratings_file(BA_path + 'ratings.txt')
print('ratings Done')
BA_reviews = parse_ratings_file(BA_path + 'reviews.txt')

ratings Done


In [42]:
#automatically remove NaN values, can be modified
def columns_to_remove(data, value, threshold):
    cols_to_remove = []
    
    for col in data.columns:
        if (data[col] == value).mean() > threshold:
            cols_to_remove.append(col)
        elif data[col].isna().mean() > threshold:
            cols_to_remove.append(col)

    return cols_to_remove

def drop_columns(data, cols_to_remove, save_path):
    data_new = data.drop(columns=cols_to_remove, errors='ignore')
    data_new.to_csv(save_path, index=False)

In [40]:
#allows application of multiple values
def column_delete_save(data, values, save_path, threshold):
    data_new = data.copy()

    for val in values:
        cols_remove = columns_to_remove(data_new, val, threshold)
        print(cols_remove)
        data_new = data_new.drop(columns=cols_remove, errors='ignore')
    
    data_new.to_csv(save_path, index=False)

In [43]:
#Test with simple df
data = {
    'A': [1, 2, 3, 4, np.NaN],
    'B': [0, 0, 0, 0, 0],
    'C': [1, 2, 1, 2, 1],
    'D': [np.NaN, np.NaN, np.NaN, np.NaN, np.NaN],
    'E': [5, 6, 7, 8, 9]
    }

df = pd.DataFrame(data)

print(df.head())

print(columns_to_remove(df, 0, 0.8))
print(columns_to_remove(df, 'NaN', 0.8))

column_delete_save(df, [0], 'test.csv', 0.8)

     A  B  C   D  E
0  1.0  0  1 NaN  5
1  2.0  0  2 NaN  6
2  3.0  0  1 NaN  7
3  4.0  0  2 NaN  8
4  NaN  0  1 NaN  9
['B', 'D']
['D']
['B', 'D']


In [44]:
#make new directories for save
def create_directory(directory_path):
    """
    Create a directory if it does not exist.

    Parameters:
    - directory_path (str): The path of the directory to create.
    """
    try:
        # Create the directory
        os.makedirs(directory_path, exist_ok=True)  # exist_ok=True avoids errors if the directory already exists
        print(f"Directory '{directory_path}' created successfully.")
    except Exception as e:
        print(f"Error creating directory: {e}")

create_directory('modData')
create_directory('modData/BeerAdvocate')
create_directory('modData/RateBeer')

Directory 'modData' created successfully.
Directory 'modData/BeerAdvocate' created successfully.
Directory 'modData/RateBeer' created successfully.


In [46]:
column_delete_save(BA_beers, [0], 'modData/BeerAdvocate/beers.csv', 0.8)
column_delete_save(BA_breweries, [0], 'modData/BeerAdvocate/breweries.csv', 0.8)
column_delete_save(BA_users, [0], 'modData/BeerAdvocate/users.csv', 0.8)


column_delete_save(RB_beers, [0], 'modData/RateBeer/beers.csv', 0.8)
column_delete_save(RB_breweries, [0], 'modData/RateBeer/breweries.csv', 0.8)
column_delete_save(RB_users, [0], 'modData/RateBeer/users.csv', 0.8)


['bros_score', 'zscore', 'nbr_matched_valid_ratings', 'avg_matched_valid_ratings']
[]
[]
['zscore', 'nbr_matched_valid_ratings', 'avg_matched_valid_ratings']
[]
[]
